In [ ]:
Notebook overview 

1. We extracted data from the “All of Us” Research Program with 1,295 cases (essential hypertension and co-occurring dementia, after excluding individuals with Parkinson’s disease, Vascular dementia, Frontotemporal dementia, Huntington’s disease, or Creutzfeldt-Jakob disease) and 45,752 controls (essential hypertension but without any neurodegenerative conditions).
2. We extracted relatedness data.
3. We extracted ancestry predictions data.
4. We took European ancestry and removed related sample ids.
5. We prepared the Phenotype and covariate file jointly.
6. We ran Genome-wide association using PLINK.2.0 and 10 principle components, sex_at_birth, current age (2024-year of birth) as covariates and age filter > 60. 

In [ ]:
#Importing the required python libraries
import pandas
import os

In [ ]:
# This query represents dataset "EHT_Dem_remove_PD_VD_FTD_Hu_CJD" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_02537279_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (380986, 380701, 379778, 441535, 378726, 43021816, 4103534, 37117145, 372241, 4092747, 443864, 4043378, 377788, 4182210, 44782771, 37312035, 35608576, 444091, 376095, 4046090, 44782763, 4228133)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (314423, 4062811, 312648, 321638, 320456, 4217486, 314103, 4302591, 320128)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (4064308, 4253363, 4204820, 36713737, 4126631, 4140090, 4177039, 441458, 4171569)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (443605, 4046090, 4047747, 37018688)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (4043378) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (40483103) 
                    AND is_standard = 1 )) criteria ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN (372241) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_02537279_person_df = pandas.read_gbq(
    dataset_02537279_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_02537279_person_df.to_csv("EHT_Dem_remove_PD_VD_FTD_Hu_CJD.csv")


In [ ]:
import pandas
import os

# This query represents dataset "EHT" for domain "person" and was generated for All of Us Controlled Tier Dataset v7
dataset_61808986_person_sql = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (312648, 320128)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_61808986_person_df = pandas.read_gbq(
    dataset_61808986_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_61808986_person_df.to_csv("EHT.csv")


In [ ]:
# All the chromosomes were downloaded from All of Us Controlled Tier Dataset v7
!gsutil -u $GOOGLE_PROJECT -m cp -r gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/acaf_threshold_v7.1/plink_bed/acaf_threshold.chr1.* .

In [ ]:
# Relatedness data was downloaded from All of Us Controlled Tier Dataset v7
!gsutil -u $GOOGLE_PROJECT -m cp -r gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/relatedness* .

In [ ]:
# Relatedness data copied into the current folder

!cp relatedness/relatedness_flagged_samples.tsv rel.tsv

In [ ]:
# Relatedness data were extracted with there FID and IID

import pandas as pd
df = pd.read_csv("rel.tsv")

# Rename the column
df.rename(columns={'sample_id': 'FID'}, inplace=True)

# Copy the values from 'FID' to 'IID'
df['IID'] = df['FID']

df.to_csv("rel1.txt", sep='\t')


In [ ]:
# Ancestry prediction data was downloaded from All of Us Controlled Tier Dataset v7

!gsutil -u $GOOGLE_PROJECT -m cp gs://fc-aou-datasets-controlled/v7/wgs/short_read/snpindel/aux/ancestry/ancestry_preds.tsv .

In [ ]:
# Ancestry prediction data helped in extraction of the principal components (PC) values

import pandas as pd

# Read the TSV file into a DataFrame
df = pd.read_csv('ancestry_preds.tsv', sep='\t')
df["pca_features"] = df["pca_features"].str[1:-1]
PCs = df["pca_features"].str.split(",", n = 16, expand = True)
PCs = PCs.astype(float)
pid = df[["research_id"]]
columns= ["PC1","PC2","PC3","PC4","PC5","PC6","PC7","PC8","PC9","PC10","PC11","PC12","PC13","PC14","PC15","PC16"]
PCs.columns = columns
PCs.head(5)
PCs_final = pd.concat([pid, PCs], axis = 1)

In [ ]:
#Load cases (essential hypertension and co-occurring dementia, after excluding individuals with Parkinson’s disease, Vascular dementia, Frontotemporal dementia, Huntington’s disease, or Creutzfeldt-Jakob disease)

dfx = pd.read_csv("EHT_Dem_remove_PD_FTD_VD_Hu_CJD.csv")

# Merge the dataframes
merged_df = PCs_final.merge(dfx, left_on='research_id', right_on='person_id')

# Filter rows where ancestry_pred is 'eur'
filtered_df = df[df['ancestry_pred'] == 'eur']

cases_df = merged_df.merge(filtered_df, on='research_id')

# Keep only the first column
df2 = cases_df1.iloc[:, [0]]

# Step 1: Copy values from 'research_id' to 'FID'
df2['FID'] = df2['research_id']

# Step 2: Rename 'research_id' to 'IID'
df2.rename(columns={'research_id': 'IID'}, inplace=True)

# Merge the dataframes
cases_df2 = df2.merge(cases_df1, left_on='IID', right_on='research_id')

cases_df4 = cases_df2.drop(columns=['research_id'])

cases_df4['Pheno']=2

df1 = pd.read_csv("EHT_Dem_remove_PD_FTD_VD_Hu_CJD.csv")

# Extract the 'person_id' column
dff1 = df1['person_id']

print("\nExtracted 'person_id' Column:")
print(dff1)

dff1.columns = ['person_id']

# Extracting 'person id' and 'gender' columns
cases_df = df1[['person_id', 'gender']]

# Merge to find common rows
matching_rows = control_df.merge(cases_df, on=list(control_df.columns))

# Remove matching rows from df
df_filtered = df[~df.isin(matching_rows.to_dict(orient='list')).all(axis=1)]

# Adding 'Pheno' column with value 1
cases_df['Pheno'] = 2

In [ ]:
#Load controls (essential hypertension but without any neurodegenerative conditions)

df2 = pd.read_csv("EHT.csv")

# Extract the 'person_id' column
dff2 = df2['person_id']

print("\nExtracted 'person_id' Column:")
print(dff2)

dff2.columns = ['person_id']

# Extracting 'person id' and 'gender' columns
control_df = df2[['person_id', 'gender']]

# Adding 'Pheno' column with value 1
control_df['Pheno'] = 1


In [ ]:
# Concatenating DataFrames along rows
concatenated_df = pd.concat([cases_df, control_df], axis=0)

concatenated_df['Pheno'].value_counts()

# Mapping gender values to numeric codes
gender_map = {'Male': 1, 'Female': 2}
concatenated_df['gender'] = concatenated_df['gender'].map(gender_map)

# Dropping rows with NaN values in the gender column (invalid gender values)
concatenated_df.dropna(subset=['gender'], inplace=True)

# Converting gender column to integer type
concatenated_df['gender'] = concatenated_df['gender'].astype(int)

In [ ]:
controls = pd.read_csv("EHT.csv")

controls1 = controls['person_id']

controls_df1 = pd.DataFrame(controls, columns=['person_id'])

# Merge the dataframes
controls_df2 = df.merge(controls_df1, left_on='research_id', right_on='person_id’)

# Filter rows where ancestry_pred is 'eur'
controls_df4 = controls_df2[controls_df2['ancestry_pred'] == 'eur']

# Merge the dataframes
controls_df5 = controls_df4.merge(PCs_final, on='research_id’)

controls_df7 = controls_df5.drop(columns=['ancestry_pred', 'probabilities', 'pca_features', 'ancestry_pred_other','person_id','PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16'])

# Step 1: Copy values from 'research_id' to 'FID'
controls_df7['FID'] = controls_df7['research_id']

# Step 2: Rename 'research_id' to 'IID'
controls_df7.rename(columns={'research_id': 'IID'}, inplace=True)

controls_df7['Pheno']=1

In [ ]:
cases = pd.read_csv("EHT_Dem_remove_PD_FTD_VD_Hu_CJD.csv")

cases1 = cases['person_id']

cases1.head()

cases_df1 = pd.DataFrame(cases1, columns=['person_id'])

# Merge the dataframes
cases_df2 = df.merge(cases_df1, left_on='research_id', right_on='person_id')

# Only European ancestry was choosen from the samples

# Filter rows where ancestry_pred is 'eur'
cases_df4 = cases_df2[cases_df2['ancestry_pred'] == 'eur']

# Merge the dataframes
cases_df5 = cases_df4.merge(PCs_final, on='research_id')

cases_df7 = cases_df5.drop(columns=['ancestry_pred', 'probabilities', 'pca_features', 'ancestry_pred_other','person_id','PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16'])

cases_df7['Pheno']=2

# Step 1: Copy values from 'research_id' to 'FID'
cases_df7['FID'] = cases_df7['research_id']

# Step 2: Rename 'research_id' to 'IID'
cases_df7.rename(columns={'research_id': 'IID'}, inplace=True)

In [ ]:
# Concatenate DataFrames
concatenated_df1 = pd.concat([cases_df7, controls_df7], ignore_index=True)

# Swapping the FID and IID columns
concatenated_df2 = concatenated_df1[['FID', 'IID', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'Pheno']]

# Merge the dataframes on the matching columns
merged_df = pd.merge(concatenated_df, concatenated_df1, left_on='person_id', right_on='IID')

# Drop the 'person_id' and 'pheno_x' columns
result_df = merged_df.drop(columns=['person_id', 'Pheno_x’])

# Rename 'pheno_x' to 'pheno'
result_df1 = result_df.rename(columns={'Pheno_y': 'Pheno'})

# Get the list of columns
columns = ['gender', 'FID', 'IID', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'Pheno']

# Move the first column to the end
columns.append(columns.pop(0))

# Reorder the dataframe
df_reordered = result_df1[columns]

final = df_reordered.drop_duplicates()

# Drop duplicates based on the IID column
final = final.drop_duplicates(subset='IID')

# Count occurrences of each value in the 'Pheno' column
pheno_counts = final['Pheno'].value_counts()

# Get the counts for specific values
count_1 = pheno_counts.get(1, 0)
count_2 = pheno_counts.get(2, 0)

print(f"Number of 1s in 'Pheno': {count_1}")
print(f"Number of 2s in 'Pheno': {count_2}")

In [ ]:
# Phenotype file containing the phenotype and covariates were jointly made

final.to_csv("Pheno_HT_Dem_Vs_HT_remove_PD_VD_FTD_Hu_CJD.txt", sep = '\t', index=False)


# Relatedness was removed from the samples

!plink2 --bfile acaf_threshold.chr2 --remove rel1.txt --make-bed --out acaf_threshold.chr2.rel.removed

# Genome-wide association analysis was done

%%bash
plink2 --bfile acaf_threshold.chr2.rel.removed --snp chr2:36408804:T:C --geno 0.05 \
        --maf 0.05 \
      --pheno Pheno_HT_Dem_Vs_HT_remove_PD_VD_FTD_Hu_CJD.txt \
      --pheno-name Pheno \
      --covar Pheno_HT_Dem_Vs_HT_remove_PD_VD_FTD_Hu_CJD.txt \
      --covar-name PC1,PC2,PC3,PC4,PC5,gender \
      --keep Pheno_HT_Dem_Vs_HT_remove_PD_VD_FTD_Hu_CJD.txt \
      --allow-no-sex \
      --glm hide-covar no-x-sex cols=+beta,+a1freq,+a1freqcc,+a1count,+totallele,+a1countcc,+totallelecc,+err \
      --out gwas_2_wgs_remove_PD_VD_FTD_Hu_CJD